### dev features

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from connectivity_mat import load_connectivity

In [2]:
df = pd.read_csv('./model_performance/output_features/feature_eng_paintype_all.csv')

,test_roc_auc_ovr
dataset,
_fa-qsidp-conn,0.566105
_fa-qsidp-conn_umap-qsidp-conn,0.566278
_kmeans-qsidp-conn,0.563260
_kmeans-qsidp-conn_fa-qsidp-conn,0.565035
_kmeans-qsidp-conn_fa-qsidp-conn_umap-qsidp-conn,0.570314
_kmeans-qsidp-conn_umap-qsidp-conn,0.567709
_quantile-qsidp-conn,0.565145
_quantile-qsidp-conn_fa-qsidp-conn,0.564649
_quantile-qsidp-conn_fa-qsidp-conn_umap-qsidp-conn,0.569944


In [2]:
df = load_connectivity(task_name='paintype_restricted', 
                       conn_type='fullcorr_100', 
                       add_questionnaire=True, 
                       add_idp=True, 
                       add_conn=True, patient_grouping='grouped')

<ipython-input-2-90bca2dfad87>:1: DtypeWarning: Columns (863,867) have mixed types.Specify dtype option on import or set low_memory=False.
  df = load_connectivity(task_name='paintype_restricted',


Questionnaires from visits [2] shape=(1708, 1029)
After imputation shape=(1708, 984)


In [3]:
df['label'].value_counts()

osteoarthritis              564
irritable bowel syndrome    438
migraine                    326
back pain                   281
Name: label, dtype: int64

In [4]:
from clean_questions import cv_classify
df_res = cv_classify(df, classifier='rforest', cv_fold=4, scaler=True, balance=True)

4
   fit_time  score_time  test_accuracy  test_f1_micro  test_roc_auc_ovr
0  1.736370    0.052992       0.313167       0.313167          0.578423
1  1.751135    0.046593       0.330961       0.330961          0.603002
2  1.721544    0.055476       0.341637       0.341637          0.567635
3  1.749790    0.048204       0.359431       0.359431          0.610711


In [15]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
X, y = load_iris(return_X_y=True)
clf = LogisticRegression(solver="liblinear").fit(X, y)
roc_auc_score(y, clf.predict_proba(X), multi_class='ovo')

0.9913333333333334

In [14]:
clf.predict_proba(X)

array([[8.78030305e-01, 1.21958900e-01, 1.07949250e-05],
       [7.97058292e-01, 2.02911413e-01, 3.02949242e-05],
       [8.51997665e-01, 1.47976480e-01, 2.58550858e-05],
       [8.23406019e-01, 1.76536159e-01, 5.78217704e-05],
       [8.96034973e-01, 1.03953836e-01, 1.11907339e-05],
       [9.26234254e-01, 7.37527845e-02, 1.29612594e-05],
       [8.94096848e-01, 1.05863935e-01, 3.92166195e-05],
       [8.60034410e-01, 1.39946715e-01, 1.88751124e-05],
       [8.01028643e-01, 1.98886755e-01, 8.46025595e-05],
       [7.92662392e-01, 2.07312003e-01, 2.56051563e-05],
       [8.90486112e-01, 1.09507726e-01, 6.16178069e-06],
       [8.61800668e-01, 1.38164963e-01, 3.43688884e-05],
       [7.85364369e-01, 2.14608265e-01, 2.73660893e-05],
       [8.33122325e-01, 1.66845600e-01, 3.20746459e-05],
       [9.27105079e-01, 7.28939564e-02, 9.65025075e-07],
       [9.64209783e-01, 3.57879589e-02, 2.25815135e-06],
       [9.40244677e-01, 5.97504819e-02, 4.84124978e-06],
       [8.90383643e-01, 1.09602

In [10]:
from sklearn.metrics import roc_auc_score
a = [1,2,3]
b = [3,3,3]
roc_auc_score(a, b, multi_class='ovo')

AxisError: axis 1 is out of bounds for array of dimension 1

In [5]:
import warnings; warnings.simplefilter('ignore')

In [3]:
qs_id = True
idp_id = True
conn_id = True

df = load_connectivity(task_name='paintype_all', 
                       conn_type='fullcorr_100',add_questionnaire=qs_id, add_idp=idp_id, add_conn=conn_id)

Questionnaires from visits [2] shape=(4900, 1029)
After imputation shape=(4900, 993)


In [4]:
df['label'].value_counts()

1    2779
0    1869
Name: label, dtype: int64

#### main split

In [5]:
def tsplit(df, test_size, random_state, scale=True):
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import confusion_matrix, classification_report
    from sklearn.model_selection import train_test_split
    from imblearn.under_sampling import RandomUnderSampler
    train, test = train_test_split(df, test_size=test_size, random_state=random_state, 
                              stratify=df['label'])
    train.drop(columns='eid', inplace=True)
    test.drop(columns='eid', inplace=True)
    cols = train.columns.to_list()
    cols.remove('label')
    
    if scale:
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler().fit(train[cols])
        train[cols] = scaler.transform(train[cols])
        test[cols] = scaler.transform(test[cols])

    return train, test

In [6]:
# balance=True
scaler=True
test_size=0.25
random_state=4

In [7]:
train, test = tsplit(df, test_size, random_state, scale=True)

In [8]:
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier

In [9]:
def print_metrics(y_true, y_pred, tag):
    from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
    if isinstance(y_true, pd.DataFrame) or isinstance(y_true, pd.Series):
        y_true = y_true.values
    if isinstance(y_pred, pd.DataFrame) or isinstance(y_pred, pd.Series):
        y_pred = y_pred.values
    if y_true.ndim>1:
        y_true=y_true.ravel()
    if y_pred.ndim>1:
        y_pred=y_pred.ravel()
    val_acc = accuracy_score(y_true, y_pred)
    val_f1 = f1_score(y_true, y_pred)
    val_auc = roc_auc_score(y_true, y_pred)
    print(f"{tag} Acc: {val_acc} \n {tag} F1: {val_f1} \n {tag} AUC: {val_auc}")

##### control condition (not scaling)

In [42]:
train, test = tsplit(df, test_size=0.25, random_state=4, scale=False)

In [45]:
target_col = 'label'
clf = lgb.LGBMClassifier(random_state=42, n_jobs=-1)
clf.fit(train.drop(columns=target_col), train[target_col].values.ravel())
test_pred = clf.predict(test.drop(columns=target_col))
print_metrics(test[target_col], test_pred, "Holdout")

Holdout Acc: 0.6419965576592083 
 Holdout F1: 0.7326478149100257 
 Holdout AUC: 0.5985087732811609


In [46]:
target_col = 'label'
clf = RandomForestClassifier(max_depth=5,random_state=42)
clf.fit(train.drop(columns=target_col), train[target_col].values.ravel())
test_pred = clf.predict(test.drop(columns=target_col))
print_metrics(test[target_col], test_pred, "Holdout")

Holdout Acc: 0.612736660929432 
 Holdout F1: 0.7434435575826681 
 Holdout AUC: 0.533304576895229


##### control condition (scaling)

In [47]:
train, test = tsplit(df, test_size=0.25, random_state=4, scale=True)

In [48]:
target_col = 'label'
clf = lgb.LGBMClassifier(random_state=42, n_jobs=-1)
clf.fit(train.drop(columns=target_col), train[target_col].values.ravel())
test_pred = clf.predict(test.drop(columns=target_col))
print_metrics(test[target_col], test_pred, "Holdout")

Holdout Acc: 0.6222030981067126 
 Holdout F1: 0.718409236690186 
 Holdout AUC: 0.5773958991265231


In [49]:
target_col = 'label'
clf = RandomForestClassifier(max_depth=5,random_state=42)
clf.fit(train.drop(columns=target_col), train[target_col].values.ravel())
test_pred = clf.predict(test.drop(columns=target_col))
print_metrics(test[target_col], test_pred, "Holdout")

Holdout Acc: 0.6135972461273667 
 Holdout F1: 0.743281875357347 
 Holdout AUC: 0.5350777194090552


#### prep qsidp vs conn columns

In [18]:
train.columns

Index([          0,           1,           2,           3,           4,
                 5,           6,           7,           8,           9,
       ...
       '25226-2.0', '25708-2.0', '25742-2.0', '25279-2.0', '25518-2.0',
       '25614-2.0', '25533-2.0', '25713-2.0', '25590-2.0',     'label'],
      dtype='object', length=2477)

In [60]:
QSIDP = [col for col in train.columns if '-' in str(col)]
CONN = [col for col in train.columns if '-' not in str(col)]
CONN.remove('label')

#### try quantile transform

In [20]:
from sklearn.preprocessing import QuantileTransformer

In [21]:
model_output_folder = './feats_out'
IS_TRAIN = True

if IS_TRAIN:
    os.makedirs(model_output_folder, exist_ok=True)

In [54]:
# train, test = tsplit(df, test_size=0.25, random_state=4, scale=True)
train, test = tsplit(df, test_size=0.25, random_state=4, scale=False)

In [55]:
train_features = []
test_features = []
for col in (CONN + QSIDP):
# for col in CONN:

    #  transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
    vec_len = len(train[col].values)
    vec_len_test = len(test[col].values)
    raw_vec = train[col].values.reshape(vec_len, 1)

    if IS_TRAIN:
        transformer = QuantileTransformer(n_quantiles=100,
                                          random_state=0,
                                          output_distribution="normal")
        transformer.fit(raw_vec)
        pd.to_pickle(transformer, f'{model_output_folder}/{col}_quantile_transformer.pkl')
    else:
        transformer = pd.read_pickle(f'{model_output_folder}/{col}_quantile_transformer.pkl')

    train[col] = transformer.transform(raw_vec)
    test[col] = transformer.transform(
        test[col].values.reshape(vec_len_test,
                                          1)).reshape(1, vec_len_test)[0]
#     train[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
#     test_features[col] = transformer.transform(
#         test[col].values.reshape(vec_len_test,
#                                           1)).reshape(1, vec_len_test)[0]

##### transform connectivity only (not scaling)

In [28]:
target_col = 'label'
clf = lgb.LGBMClassifier(random_state=42, n_jobs=-1)
clf.fit(train.drop(columns=target_col), train[target_col].values.ravel())
test_pred = clf.predict(test.drop(columns=target_col))
print_metrics(test[target_col], test_pred, "Holdout")

Holdout Acc: 0.6273666092943201 
 Holdout F1: 0.7226137091607944 
 Holdout AUC: 0.5824149245913761


In [29]:
target_col = 'label'
clf = RandomForestClassifier(max_depth=5,random_state=42)
clf.fit(train.drop(columns=target_col), train[target_col].values.ravel())
test_pred = clf.predict(test.drop(columns=target_col))
print_metrics(test[target_col], test_pred, "Holdout")

Holdout Acc: 0.612736660929432 
 Holdout F1: 0.7434435575826681 
 Holdout AUC: 0.533304576895229


##### transform connectivity only ( scaling)

In [32]:
target_col = 'label'
clf = lgb.LGBMClassifier(random_state=42, n_jobs=-1)
clf.fit(train.drop(columns=target_col), train[target_col].values.ravel())
test_pred = clf.predict(test.drop(columns=target_col))
print_metrics(test[target_col], test_pred, "Holdout")

Holdout Acc: 0.6161790017211703 
 Holdout F1: 0.7137355584082158 
 Holdout AUC: 0.5713062098501072


In [33]:
target_col = 'label'
clf = RandomForestClassifier(max_depth=5,random_state=42)
clf.fit(train.drop(columns=target_col), train[target_col].values.ravel())
test_pred = clf.predict(test.drop(columns=target_col))
print_metrics(test[target_col], test_pred, "Holdout")

Holdout Acc: 0.6135972461273667 
 Holdout F1: 0.743281875357347 
 Holdout AUC: 0.5350777194090552


##### transform everything (not scaling)

In [56]:
target_col = 'label'
clf = lgb.LGBMClassifier(random_state=42, n_jobs=-1)
clf.fit(train.drop(columns=target_col), train[target_col].values.ravel())
test_pred = clf.predict(test.drop(columns=target_col))
print_metrics(test[target_col], test_pred, "Holdout")

Holdout Acc: 0.6316695352839932 
 Holdout F1: 0.7242268041237114 
 Holdout AUC: 0.588470722351455


In [57]:
target_col = 'label'
clf = RandomForestClassifier(max_depth=5,random_state=42)
clf.fit(train.drop(columns=target_col), train[target_col].values.ravel())
test_pred = clf.predict(test.drop(columns=target_col))
print_metrics(test[target_col], test_pred, "Holdout")

Holdout Acc: 0.6135972461273667 
 Holdout F1: 0.743281875357347 
 Holdout AUC: 0.5350777194090552


##### transform everything (scaling)

In [52]:
target_col = 'label'
clf = lgb.LGBMClassifier(random_state=42, n_jobs=-1)
clf.fit(train.drop(columns=target_col), train[target_col].values.ravel())
test_pred = clf.predict(test.drop(columns=target_col))
print_metrics(test[target_col], test_pred, "Holdout")

Holdout Acc: 0.6316695352839932 
 Holdout F1: 0.7242268041237114 
 Holdout AUC: 0.588470722351455


In [53]:
target_col = 'label'
clf = RandomForestClassifier(max_depth=5,random_state=42)
clf.fit(train.drop(columns=target_col), train[target_col].values.ravel())
test_pred = clf.predict(test.drop(columns=target_col))
print_metrics(test[target_col], test_pred, "Holdout")

Holdout Acc: 0.6135972461273667 
 Holdout F1: 0.743281875357347 
 Holdout AUC: 0.5350777194090552


#### try factor analysis

In [84]:
# train, test = tsplit(df, test_size, random_state, scale=True)
train, test = tsplit(df, test_size, random_state, scale=False)

In [85]:
from sklearn.decomposition import PCA,FactorAnalysis

n_comp = 50

data = pd.concat(
    [pd.DataFrame(train[CONN]),
     pd.DataFrame(test[CONN])])
#data2 = (FactorAnalysis(n_components=n_comp, random_state=42).fit_transform(data[GENES]))
if IS_TRAIN:
    fa = FactorAnalysis(n_components=n_comp,
                        random_state=1903).fit(data[CONN])
    pd.to_pickle(fa, f'{model_output_folder}/factor_analysis_c.pkl')
    #umap = UMAP(n_components=n_dim, random_state=1903).fit(data[GENES])
    #pd.to_pickle(umap, f'{MODEL_DIR}/{NB}_umap_g.pkl')
else:
    fa = pd.read_pickle(f'{model_output_folder}/factor_analysis_c.pkl')
    #umap = pd.read_pickle(f'{MODEL_DIR}/{NB}_umap_g.pkl')
data2 = fa.transform(data[CONN])
train2 = data2[:train.shape[0]]
test2 = data2[-test.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca-c-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca-c-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train = pd.concat((train.reset_index(drop=True), train2.reset_index(drop=True)), axis=1)
test = pd.concat((test.reset_index(drop=True), test2.reset_index(drop=True)), axis=1)

In [62]:
# conn only (scaled)
target_col = 'label'
clf = RandomForestClassifier(max_depth=5,random_state=42)
clf.fit(train.drop(columns=target_col), train[target_col].values.ravel())
test_pred = clf.predict(test.drop(columns=target_col))
print_metrics(test[target_col], test_pred, "Holdout")

Holdout Acc: 0.6049913941480206 
 Holdout F1: 0.7360552041403106 
 Holdout AUC: 0.5278834748047387


In [65]:
# conn only (not scaled)
target_col = 'label'
clf = RandomForestClassifier(max_depth=5,random_state=42)
clf.fit(train.drop(columns=target_col), train[target_col].values.ravel())
test_pred = clf.predict(test.drop(columns=target_col))
print_metrics(test[target_col], test_pred, "Holdout")

Holdout Acc: 0.6049913941480206 
 Holdout F1: 0.7369627507163323 
 Holdout AUC: 0.5268297567513441


In [74]:
# train, test = tsplit(df, test_size, random_state, scale=True)
train, test = tsplit(df, test_size, random_state, scale=False)

In [86]:
from sklearn.decomposition import PCA,FactorAnalysis

n_comp = 50

data = pd.concat(
    [pd.DataFrame(train[QSIDP]),
     pd.DataFrame(test[QSIDP])])
#data2 = (FactorAnalysis(n_components=n_comp, random_state=42).fit_transform(data[GENES]))
if IS_TRAIN:
    fa = FactorAnalysis(n_components=n_comp,
                        random_state=1903).fit(data[QSIDP])
    pd.to_pickle(fa, f'{model_output_folder}/factor_analysis_q.pkl')
    #umap = UMAP(n_components=n_dim, random_state=1903).fit(data[GENES])
    #pd.to_pickle(umap, f'{MODEL_DIR}/{NB}_umap_g.pkl')
else:
    fa = pd.read_pickle(f'{model_output_folder}/factor_analysis_q.pkl')
    #umap = pd.read_pickle(f'{MODEL_DIR}/{NB}_umap_g.pkl')
data2 = fa.transform(data[QSIDP])
train2 = data2[:train.shape[0]]
test2 = data2[-test.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca-q-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca-q-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train = pd.concat((train.reset_index(drop=True), train2.reset_index(drop=True)), axis=1)
test = pd.concat((test.reset_index(drop=True), test2.reset_index(drop=True)), axis=1)

In [68]:
# qsidp only (scaled)
target_col = 'label'
clf = RandomForestClassifier(max_depth=5,random_state=42)
clf.fit(train.drop(columns=target_col), train[target_col].values.ravel())
test_pred = clf.predict(test.drop(columns=target_col))
print_metrics(test[target_col], test_pred, "Holdout")

Holdout Acc: 0.5938037865748709 
 Holdout F1: 0.7293577981651376 
 Holdout AUC: 0.5153698026589435


In [71]:
# qsidp only (not scaled)
target_col = 'label'
clf = RandomForestClassifier(max_depth=5,random_state=42)
clf.fit(train.drop(columns=target_col), train[target_col].values.ravel())
test_pred = clf.predict(test.drop(columns=target_col))
print_metrics(test[target_col], test_pred, "Holdout")

Holdout Acc: 0.6015490533562823 
 Holdout F1: 0.7358813462635482 
 Holdout AUC: 0.5218446228028284


In [78]:
# conn and qsidp (not scaled)
target_col = 'label'
clf = RandomForestClassifier(max_depth=5,random_state=42)
clf.fit(train.drop(columns=target_col), train[target_col].values.ravel())
test_pred = clf.predict(test.drop(columns=target_col))
print_metrics(test[target_col], test_pred, "Holdout")

Holdout Acc: 0.6049913941480206 
 Holdout F1: 0.7390562819783968 
 Holdout AUC: 0.5243710812934235


In [82]:
# conn and qsidp (scaled)
target_col = 'label'
clf = RandomForestClassifier(max_depth=5,random_state=42)
clf.fit(train.drop(columns=target_col), train[target_col].values.ravel())
test_pred = clf.predict(test.drop(columns=target_col))
print_metrics(test[target_col], test_pred, "Holdout")

Holdout Acc: 0.6058519793459552 
 Holdout F1: 0.7391799544419134 
 Holdout AUC: 0.5257929844561182


In [83]:
# conn and qsidp (scaled)
target_col = 'label'
clf = lgb.LGBMClassifier(random_state=42, n_jobs=-1)
clf.fit(train.drop(columns=target_col), train[target_col].values.ravel())
test_pred = clf.predict(test.drop(columns=target_col))
print_metrics(test[target_col], test_pred, "Holdout")

Holdout Acc: 0.6445783132530121 
 Holdout F1: 0.7347463070006421 
 Holdout AUC: 0.6010182860135874


In [88]:
# conn and qsidp (not scaled)
target_col = 'label'
clf = lgb.LGBMClassifier(random_state=42, n_jobs=-1)
clf.fit(train.drop(columns=target_col), train[target_col].values.ravel())
test_pred = clf.predict(test.drop(columns=target_col))
print_metrics(test[target_col], test_pred, "Holdout")

Holdout Acc: 0.6230636833046471 
 Holdout F1: 0.721728081321474 
 Holdout AUC: 0.5756566481290342


#### cluster

In [ ]:
from sklearn.cluster import KMeans
def fe_cluster(train, test, n_clusters_g = 45, n_clusters_c = 15, SEED = 123):
    
    features_g = list(train.columns[4:776])
    features_c = list(train.columns[776:876])
    
    def create_cluster(train, test, features, kind = 'g', n_clusters = n_clusters_g):
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis = 0)
        kmeans = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
        train[f'clusters_{kind}'] = kmeans.labels_[:train.shape[0]]
        test[f'clusters_{kind}'] = kmeans.labels_[train.shape[0]:]
        train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
        test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
        return train, test
    
    train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = n_clusters_g)
    train, test = create_cluster(train, test, features_c, kind = 'c', n_clusters = n_clusters_c)
    return train, test

train_features ,test_features=fe_cluster(train_features,test_features)

In [89]:
train.columns

Index([         0,          1,          2,          3,          4,          5,
                6,          7,          8,          9,
       ...
       'pca-q-40', 'pca-q-41', 'pca-q-42', 'pca-q-43', 'pca-q-44', 'pca-q-45',
       'pca-q-46', 'pca-q-47', 'pca-q-48', 'pca-q-49'],
      dtype='object', length=2577)